In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

Update with new data

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm

# --- 1. Initial data ---
X_init = np.array([
    [0.31940389, 0.76295937],
    [0.57432921, 0.8798981],
    [0.73102363, 0.73299988],
    [0.84035342, 0.26473161],
    [0.65011406, 0.68152635],
    [0.41043714, 0.1475543],
    [0.31269116, 0.07872278],
    [0.68341817, 0.86105746],
    [0.08250725, 0.40348751],
    [0.88388983, 0.58225397]
])

y_init = np.array([
    1.32267704e-079,  1.03307824e-046,  7.71087511e-016,
    3.34177101e-124, -3.60606264e-003, -2.15924904e-054,
    -2.08909327e-091,  2.53500115e-040,  3.60677119e-081,
    6.22985647e-048
])

In [ ]:
# Add the new observation
X_new = np.array([[0.080808, 0.404040]])
y_new = np.array([5.34214011784672e-82])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

In [ ]:
print(X_init)
print(X_init.shape)   # Useful if it’s an array
print(type(X_init)) 

In [ ]:
print(y_init)
print(y_init.shape)   # Useful if it’s an array
print(type(y_init)) 

# Week 1 UCB method


In [ ]:
# Optional: Log-transform to handle sparse/near-zero outputs
#y_trans = np.log(np.abs(y_init) + 1e-8)
y_trans = y_init.copy()  # Use raw outputs


# --- 2. Fit Gaussian Process ---
#kernel = Matern(nu=2.5)
#gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)

kernel = Matern(length_scale=0.1, nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)

gp.fit(X_init, y_trans)

# --- 3. Define UCB acquisition function ---
def acquisition_ucb(X, gp, kappa=2.0):
    mu, sigma = gp.predict(X, return_std=True)
    return mu + kappa * sigma

# --- 4. Generate candidate points on a 2D grid ---
grid_size = 100
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
X_candidates = np.array([[i, j] for i in x1 for j in x2])

# --- 5. Evaluate acquisition function ---
acq_values = acquisition_ucb(X_candidates, gp, kappa=2.5)

# --- 6. Select next point ---
next_point = X_candidates[np.argmax(acq_values)]
print("Next point to query:", next_point)


In [ ]:
# Stabilize and preserve sign (for small negative readings)
y_trans = np.sign(y_all) * np.log10(np.abs(y_all) + 1e-20)


In [ ]:
kernel = Matern(length_scale=0.15, nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_trans)

###
#Note: slightly increase length_scale — your previous grid suggests spatial correlation extends across ~0.1–0.2 units.
###

# Choose a smarter acquisition strategy
Bootstrapping is a resampling method where you generate multiple samples with replacement from your original data. You then calculate the statistic (mean, median, correlation, etc.) on each sample to understand the variability and uncertainty of that statistic.

**Scenario**
UCB (mu + κσ) is good for exploration, but since I have mostly near-zero outputs, we may want to mix in exploration and exploitation adaptively.

Try using Expected Improvement (EI) instead of UCB.
It’s more focused on discovering true peaks when most readings are near noise.

In [ ]:
from scipy.stats import norm

def acquisition_ei(X, gp, y_best, xi=0.01):
    mu, sigma = gp.predict(X, return_std=True)
    sigma = sigma.reshape(-1, 1)
    mu = mu.reshape(-1, 1)

    imp = mu - y_best - xi
    Z = imp / (sigma + 1e-9)
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei.ravel()


In [ ]:
# Generate 2D grid of candidates
grid_size = 100
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
X_candidates = np.array([[i, j] for i in x1 for j in x2])

# Best observed value
y_best = np.max(y_trans)

# Compute acquisition values
acq_values = acquisition_ei(X_candidates, gp, y_best, xi=0.02)

# Select next point
next_point = X_candidates[np.argmax(acq_values)]
print("Next point to query:", next_point)
